<a href="https://colab.research.google.com/github/Maucho6340/Telecom_X_LATAM_Predict/blob/main/Challenge_predict_evasion_op1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='blue'> CHALLENGE TELECOM X:
**An√°lisis de evasi√≥n de clientes Parte-2**

## **MISI√ìN**



Tu nueva misi√≥n es desarrollar modelos predictivos capaces de prever qu√© clientes tienen mayor probabilidad de cancelar sus servicios.

La empresa quiere anticiparse al problema de la cancelaci√≥n, y te corresponde a ti construir un pipeline robusto para esta etapa inicial de modelado.

## **OBJETIVOS DEL DESAF√çO**



- Preparar los datos para el modelado (tratamiento, codificaci√≥n, normalizaci√≥n).

- Realizar an√°lisis de correlaci√≥n y selecci√≥n de variables.

- Entrenar dos o m√°s modelos de clasificaci√≥n.

- Evaluar el rendimiento de los modelos con m√©tricas.

- Interpretar los resultados, incluyendo la importancia de las variables.

- Crear una conclusi√≥n estrat√©gica se√±alando los principales factores que influyen en la cancelaci√≥n.

## **DESARROLLO**

### **Verificar versiones de bibliotecas**

In [25]:
import pandas, numpy, matplotlib, seaborn, sklearn, scipy #scipy
print('Versi√≥n de Pandas: %s' % pandas.__version__)
print('Versi√≥n de Numpy: %s' % numpy.__version__)
print('Versi√≥n de Matplotlib: %s' % matplotlib.__version__)
print('Versi√≥n de Seaborn: %s' % seaborn.__version__)
print('Versi√≥n de Sklearn: %s' % sklearn.__version__)
print('Versi√≥n de Scipy: %s' % scipy.__version__)

Versi√≥n de Pandas: 2.2.2
Versi√≥n de Numpy: 2.0.2
Versi√≥n de Matplotlib: 3.10.0
Versi√≥n de Seaborn: 0.13.2
Versi√≥n de Sklearn: 1.6.1
Versi√≥n de Scipy: 1.16.1


### **Cargar archivo**

In [26]:
import pandas as pd

# Cargar archivo CSV
df_final = pd.read_csv("/content/df_predict_churn_2.csv")

# Vista r√°pida de los primeros registros
df_final.head(5)

ID_Cliente Churn customer.gender  customer.SeniorCitizen customer.Partner  \
0  0002-ORFBO    No          Female                       0              Yes   
1  0003-MKNFE    No            Male                       0               No   
2  0004-TLHLJ   Yes            Male                       0               No   
3  0011-IGKFF   Yes            Male                       1              Yes   
4  0013-EXCHZ   Yes          Female                       1              Yes   

  customer.Dependents  customer.tenure phone.PhoneService phone.MultipleLines  \
0                 Yes                9                Yes                  No   
1                  No                9                Yes                 Yes   
2                  No                4                Yes                  No   
3                  No               13                Yes                  No   
4                  No                3                Yes                  No   

  internet.InternetService  ... internet.StreamingTV internet.StreamingMovies  \
0                      DSL  ...                  Yes                       No   
1                      DSL  ...                   No                      Yes   
2             Fibra √ìptica  ...                   No                       No   
3             Fibra √ìptica  ...                  Yes                      Yes   
4             Fibra √ìptica  ...                  Yes                       No   

  account.Contract account.PaperlessBilling account.PaymentMethod  \
0         One year                      Yes          Mailed check   
1   Month-to-month                       No          Mailed check   
2   Month-to-month                      Yes      Electronic check   
3   Month-to-month                      Yes      Electronic check   
4   Month-to-month                      Yes          Mailed check   

  Facturacion_Mensual account.Charges.Total Cuentas_Diarias Churn_bin  \
0                65.6                593.30        2.186667       0.0   
1                59.9                542.40        1.996667       0.0   
2                73.9                280.85        2.463333       1.0   
3                98.0               1237.85        3.266667       1.0   
4                83.9                267.40        2.796667       1.0   

   Churn_num  
0        0.0  
1        0.0  
2        1.0  
3        1.0  
4        1.0  

[5 rows x 24 columns]

In [27]:
#
#
#
# Renombrar columnas que contienen 'Churn' por 'Chrun'
df_final.columns = [
    col.replace('Churn', 'Chrun') if 'Churn' in col else col
    for col in df_final.columns
]

# Verificar el cambio
print(df_final.columns[df_final.columns.str.contains('Chrun')])
#
#
#
#

Index(['Chrun', 'Chrun_bin', 'Chrun_num'], dtype='object')


In [28]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID_Cliente                 7267 non-null   object 
 1   Chrun                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

En este caso, como el objetivo es predecir si un cliente va a cancelar o no el servicio, estamos trabajando con una variable categ√≥rica (churn: s√≠ o no). Por eso, los modelos adecuados son los de clasificaci√≥n, como DecisionTreeClassifier o KNeighborsClassifier

In [29]:
df_final.sample(5)

ID_Cliente Chrun customer.gender  customer.SeniorCitizen  \
2517  3518-FSTWG    No            Male                       1   
3139  4374-YMUSQ    No            Male                       0   
5165  7048-GXDAY    No            Male                       0   
5675  7754-IXRMC    No          Female                       0   
2827  3920-HIHMQ    No          Female                       0   

     customer.Partner customer.Dependents  customer.tenure phone.PhoneService  \
2517              Yes                  No               72                Yes   
3139               No                  No               34                Yes   
5165               No                  No               39                Yes   
5675               No                  No                1                Yes   
2827               No                 Yes               26                Yes   

     phone.MultipleLines internet.InternetService  ... internet.StreamingTV  \
2517                 Yes             Fibra √ìptica  ...                  Yes   
3139                 Yes             Fibra √ìptica  ...                  Yes   
5165                 Yes             Fibra √ìptica  ...                  Yes   
5675                  No                      DSL  ...                   No   
2827                 Yes                      DSL  ...                   No   

     internet.StreamingMovies account.Contract account.PaperlessBilling  \
2517                      Yes         Two year                      Yes   
3139                      Yes         One year                       No   
5165                      Yes   Month-to-month                      Yes   
5675                       No   Month-to-month                       No   
2827                       No   Month-to-month                      Yes   

          account.PaymentMethod Facturacion_Mensual account.Charges.Total  \
2517  Bank transfer (automatic)              109.55               7920.70   
3139    Credit card (automatic)              116.25               3899.05   
5165           Electronic check              106.40               4040.65   
5675               Mailed check               45.35                 45.35   
2827               Mailed check               49.50               1265.65   

     Cuentas_Diarias Chrun_bin  Chrun_num  
2517        3.651667       0.0        0.0  
3139        3.875000       0.0        0.0  
5165        3.546667       0.0        0.0  
5675        1.511667       0.0        0.0  
2827        1.650000       0.0        0.0  

[5 rows x 24 columns]

### <font color='blue'>**1.- Preparar los datos para el modelado (tratamiento, codificaci√≥n, normalizaci√≥n)**.

##### **Tratamiento de valores nulos**

In [30]:
# Imputar con mediana en 'account.Charges.Total' ‚Äî versi√≥n segura
df_final['account.Charges.Total'] = df_final['account.Charges.Total'].fillna(df_final['account.Charges.Total'].median())

# Eliminar filas sin datos en la variable objetivo
df_final.dropna(subset=['Chrun_bin', 'Chrun_num'], inplace=True)

##### **Codificaci√≥n de Variables Categ√≥rica**

In [31]:
# Elimina columna ID_Cliente o l ignora si ya no existe
df_final.drop(columns=['ID_Cliente'], inplace=True, errors='ignore')

# Codificaci√≥n con pd.get_dummies
columnas_categoricas = df_final.select_dtypes(include='object').columns

df_final = pd.get_dummies(df_final, columns=columnas_categoricas, drop_first=True)

##### **Verificaci√≥n final del DataFram**

In [32]:
# Confirmar que no quedan datos nulos ni columnas tipo object
print(df_final.isnull().sum())
print(df_final.dtypes)

customer.SeniorCitizen                           0
customer.tenure                                  0
Facturacion_Mensual                              0
account.Charges.Total                            0
Cuentas_Diarias                                  0
Chrun_bin                                        0
Chrun_num                                        0
Chrun_Yes                                        0
customer.gender_Male                             0
customer.Partner_Yes                             0
customer.Dependents_Yes                          0
phone.PhoneService_Yes                           0
phone.MultipleLines_No phone service             0
phone.MultipleLines_Yes                          0
internet.InternetService_Fibra √ìptica            0
internet.InternetService_Sin Servicio            0
internet.OnlineSecurity_No internet service      0
internet.OnlineSecurity_Yes                      0
internet.OnlineBackup_No internet service        0
internet.OnlineBackup_Yes     

#### **Resumen 1a Etapa**

Esta etapa se centr√≥ en garantizar la calidad, consistencia y estructura adecuada del dataset para su uso en modelos de machine learning.

##### Acciones realizadas:

- **Carga del archivo**: Se verific√≥ la estructura del DataFrame, compuesto por 7,267 registros y 24 columnas.
- **Tratamiento de valores nulos**: Se imputaron o eliminaron valores faltantes, especialmente en `account.Charges.Total`, `Churn_bin` y `Churn_num`.
- **Codificaci√≥n de variables categ√≥ricas**: Se aplic√≥ One-Hot Encoding a variables como g√©nero, tipo de contrato, servicios de internet, m√©todo de pago, etc.
- **Verificaci√≥n final del DataFrame**:
  - Todas las variables relevantes quedaron sin valores nulos.
  - Las variables booleanas fueron correctamente codificadas como `bool`.
  - Las variables num√©ricas (`customer.tenure`, `Facturacion_Mensual`, `account.Charges.Total`, `Cuentas_Diarias`) quedaron listas para an√°lisis estad√≠stico.



**Conclusi√≥n de la Etapa**<br>

Se complet√≥ exitosamente la etapa de exploraci√≥n y limpieza del conjunto de datos, abordando valores faltantes, inconsistencias y codificaci√≥n de variables categ√≥ricas. Este proceso permiti√≥ establecer una base estructurada y confiable para el an√°lisis posterior. Las decisiones tomadas se documentaron cuidadosamente, priorizando la reproducibilidad y la trazabilidad del flujo de trabajo. El dataset resultante presenta una estructura coherente, libre de anomal√≠as cr√≠ticas, y est√° listo para avanzar hacia la selecci√≥n de variables relevantes.

### <font color='blue'>**2.- Realizar an√°lisis de correlaci√≥n y selecci√≥n de variables**.

##### **An√°lisis de correlaci√≥n num√©rica**

In [33]:
import seaborn as sns
import matplotlib.pyplot as plt

# Correlaci√≥n de variables num√©ricas
correlaciones = df_final.corr(numeric_only=True)

# Visualizar correlaci√≥n con la variable objetivo
plt.figure(figsize=(10, 6))
sns.heatmap(correlaciones[['Chrun_num']].sort_values(by='Chrun_num', ascending=False),
            annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlaci√≥n de variables num√©ricas con Churn_num")
plt.tight_layout()
plt.show()

##### **An√°lisis de Importancia para Variables Booleana**

In [34]:
from sklearn.tree import DecisionTreeClassifier
X = df_final.drop(columns=['Chrun_bin', 'Chrun_num', 'Chrun_Yes'])  # excluimos el target y su duplicado
y = df_final['Chrun_bin']  # o 'Churn_num' seg√∫n tu criterio

modelo_arbol = DecisionTreeClassifier(random_state=42)
modelo_arbol.fit(X, y)

importancias = pd.Series(modelo_arbol.feature_importances_, index=X.columns)
importancias_ordenadas = importancias.sort_values(ascending=False)

# Visualizamos nuevamente
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.barplot(x=importancias_ordenadas[:15], y=importancias_ordenadas.index[:15], palette='viridis')
plt.title("Top 15 variables m√°s importantes (sin Chrun_Yes)")
plt.xlabel("Importancia")
plt.ylabel("Variable")
plt.tight_layout()
plt.show()

/tmp/ipython-input-1173703578.py:16: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=importancias_ordenadas[:15], y=importancias_ordenadas.index[:15], palette='viridis')


Variables Predictivas Seleccionadas para el Modelo **DecisionTreeClassifier**

| Variable                                  | Descripci√≥n y Justificaci√≥n |
|--------------------------------------------|------------------------------|
| `customer.tenure`                          | Antig√ºedad del cliente; menor permanencia se asocia a mayor churn. |
| `account.Charges.Total`                    | Total facturado; refleja exposici√≥n financiera acumulada. |
| `internet.InternetService_Fibra √ìptica`    | Este tipo de servicio est√° correlacionado con mayor abandono. |
| `Cuentas_Diarias`                          | Actividad operativa diaria; puede influir en la permanencia. |
| `Facturacion_Mensual`                      | Costo mensual del servicio; niveles altos pueden generar abandono. |
| `customer.SeniorCitizen`                   | Grupo demogr√°fico con patrones de churn distintos. |
| `account.PaymentMethod_Electronic check`   | M√©todo de pago vinculado a mayor volatilidad de permanencia. |
| `account.Contract_Two year`                | Contrato largo suele estar ligado a menor churn. |
| `internet.OnlineSecurity_Yes`              | Servicios adicionales pueden aumentar la percepci√≥n de valor. |
| `phone.MultipleLines_Yes`                  | Presencia de l√≠neas m√∫ltiples podr√≠a asociarse a mayor complejidad o costos. |

##### **An√°lisis de Multicolinealidad con VIF**

In [35]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import pandas as pd

# Usamos df_final directamente con las variables seleccionadas
X = df_final[['customer.tenure', 'account.Charges.Total', 'Cuentas_Diarias', 'Facturacion_Mensual']]
X_const = add_constant(X)

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_const.values, i+1) for i in range(len(X.columns))]

print(vif_data)

                Variable       VIF
0        customer.tenure  5.769597
1  account.Charges.Total  9.395018
2        Cuentas_Diarias       inf
3    Facturacion_Mensual       inf


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


##### **Visualizaci√≥n de correlaciones cruzadas**

In [36]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.heatmap(X.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlaci√≥n entre Variables Predictivas")
plt.show()

##### **Comparaci√≥n con otras t√©cnicas de selecci√≥n autom√°tica**

<u>Recursive Feature Elimination (RFE)</u>

In [37]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)
rfe = RFE(model, n_features_to_select=5)
rfe.fit(X, y)

selected_features = X.columns[rfe.support_]
print("Variables seleccionadas por RFE:", selected_features.tolist())

Variables seleccionadas por RFE: ['customer.tenure', 'account.Charges.Total', 'Cuentas_Diarias', 'Facturacion_Mensual']


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_rfe.py:300: UserWarning: Found n_features_to_select=5 > n_features=4. There will be no feature selection and all features will be kept.
  warnings.warn(


<u>SelectFromModel</u>

In [38]:
from sklearn.feature_selection import SelectFromModel

sfm = SelectFromModel(model)
sfm.fit(X, y)

selected_features_sfm = X.columns[sfm.get_support()]
print("Variables seleccionadas por SelectFromModel:", selected_features_sfm.tolist())

Variables seleccionadas por SelectFromModel: ['account.Charges.Total']


##### **Resumen Etapa 2**

Esta etapa tuvo como objetivo identificar las variables m√°s relevantes para la predicci√≥n de chrun, reducir la dimensionalidad y evitar redundancias.

### An√°lisis de correlaci√≥n num√©rica

Se exploraron relaciones entre variables continuas. Se detect√≥ una correlaci√≥n casi perfecta entre `Facturacion_Mensual` y `Cuentas_Diarias`, lo que sugiere redundancia informativa.

### An√°lisis de importancia para variables booleanas

Se entren√≥ un `DecisionTreeClassifier` para obtener la importancia relativa de las variables categ√≥ricas codificadas. Se visualizaron las 15 variables m√°s influyentes en la predicci√≥n de chrun.

### An√°lisis de multicolinealidad con VIF

Se calcul√≥ el **Factor de Inflaci√≥n de Varianza (VIF)** para las variables num√©ricas seleccionadas:

| Variable                | VIF        | Interpretaci√≥n                        |
|------------------------|------------|---------------------------------------|
| `customer.tenure`      | 5.77       | Aceptable                             |
| `account.Charges.Total`| 9.40       | Alta multicolinealidad                |
| `Cuentas_Diarias`      | ‚àû          | Multicolinealidad extrema             |
| `Facturacion_Mensual`  | ‚àû          | Multicolinealidad extrema             |

Este an√°lisis confirm√≥ la necesidad de revisar la inclusi√≥n simult√°nea de Facturacion_Mensual y Cuentas_Diarias en modelos sensibles a colinealidad.
Los valores de VIF tienden a infinito debido a la correlaci√≥n perfecta entre ambas variables (coeficiente de correlaci√≥n = 1.00).


### üîπ Visualizaci√≥n de correlaciones cruzadas

Se generaron heatmaps para variables categ√≥ricas codificadas y variables num√©ricas, lo que permiti√≥ validar visualmente las relaciones detectadas por VIF y el √°rbol de decisi√≥n.

### üîπ Comparaci√≥n con t√©cnicas de selecci√≥n autom√°tica

Se aplicaron dos m√©todos complementarios:

- **RFE (Recursive Feature Elimination)**: Seleccion√≥ las siguientes variables:
  ```python
  ['customer.tenure', 'account.Charges.Total', 'Cuentas_Diarias', 'Facturacion_Mensual']
- **'SelectFromModel'**: Seleccion√≥ la siguiente variable:
   ['account.Charges.Total']

**Conclusi√≥n de la Etapa**<br>

Se consolid√≥ un conjunto de cuatro variables altamente relevantes para el modelado, seleccionadas mediante t√©cnicas autom√°ticas y an√°lisis interpretativo. Aunque se detect√≥ multicolinealidad entre dos de ellas, se decidi√≥ mantenerlas temporalmente para evaluar su impacto en el rendimiento de los modelos en la siguiente etapa. La convergencia entre m√©todos como RFE, SelectFromModel y el an√°lisis de importancia del √°rbol de decisi√≥n aporta solidez y confianza al proceso de preparaci√≥n


### <font color='blue'>**3.- Entrenar dos o m√°s modelos de clasificaci√≥n**.

In [69]:
# üì¶ Importar librer√≠as
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# üéØ Divisi√≥n inicial del dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# ‚öñÔ∏è Rebalanceo con SMOTE (solo sobre el conjunto de entrenamiento)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# üìè Escalado de variables
scaler = StandardScaler()
X_train_res_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)  # Solo transformamos, no fit

# üîÅ Validaci√≥n cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# üß™ Definici√≥n de modelos
modelo_lr = LogisticRegression(solver='liblinear', class_weight='balanced', max_iter=1000, random_state=42)
modelo_rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# üìä Validaci√≥n cruzada con F1-score
scores_lr = cross_val_score(modelo_lr, X_train_res_scaled, y_train_res, cv=cv, scoring='f1')
scores_rf = cross_val_score(modelo_rf, X_train_res_scaled, y_train_res, cv=cv, scoring='f1')

# üìà Resultados
print("F1-score promedio (Logistic Regression):", scores_lr.mean())
print("F1-score promedio (Random Forest):", scores_rf.mean())

F1-score promedio (Logistic Regression): 0.8328808393600422
F1-score promedio (Random Forest): 0.8499441424669371


### <font color='blue'>**4.- Evaluaci√≥n del redimiento de los modelos**.

##### valuaci√≥n sobre X_test_scaled y y_test
Importar m√©trica


In [70]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

##### Entrenar modelos finales sobre todo el conjunto rebalanceado

In [71]:
# Entrenar sobre todo el conjunto rebalanceado y escalado
modelo_lr.fit(X_train_res_scaled, y_train_res)
modelo_rf.fit(X_train_res_scaled, y_train_res)

RandomForestClassifier(class_weight='balanced', random_state=42)

##### Predicciones sobre el conjunto de prueba

In [72]:
y_pred_lr = modelo_lr.predict(X_test_scaled)
y_pred_rf = modelo_rf.predict(X_test_scaled)

##### M√©tricas para cada molo

In [74]:
def evaluar_modelo(y_true, y_pred, nombre):
    print(f"\nEvaluaci√≥n para {nombre}")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1-score:", f1_score(y_true, y_pred))
    print("Matriz de confusi√≥n:\n", confusion_matrix(y_true, y_pred))
    print("Reporte de clasificaci√≥n:\n", classification_report(y_true, y_pred))

# Evaluar ambos modelos
evaluar_modelo(y_test, y_pred_lr, "Logistic Regression")
evaluar_modelo(y_test, y_pred_rf, "Random Forest")


Evaluaci√≥n para Logistic Regression
Accuracy: 0.7743080198722498
Precision: 0.5689655172413793
Recall: 0.6176470588235294
F1-score: 0.5923076923076923
Matriz de confusi√≥n:
 [[860 175]
 [143 231]]
Reporte de clasificaci√≥n:
               precision    recall  f1-score   support

         0.0       0.86      0.83      0.84      1035
         1.0       0.57      0.62      0.59       374

    accuracy                           0.77      1409
   macro avg       0.71      0.72      0.72      1409
weighted avg       0.78      0.77      0.78      1409


Evaluaci√≥n para Random Forest
Accuracy: 0.7700496806245565
Precision: 0.5651041666666666
Recall: 0.5802139037433155
F1-score: 0.5725593667546174
Matriz de confusi√≥n:
 [[868 167]
 [157 217]]
Reporte de clasificaci√≥n:
               precision    recall  f1-score   support

         0.0       0.85      0.84      0.84      1035
         1.0       0.57      0.58      0.57       374

    accuracy                           0.77      1409
   macr

**Interpretaci√≥n t√©cnica**<br>
Ambos modelos tienen rendimiento similar, con una ligera ventaja para Logistic Regression en todas las m√©tricas.<br>
La clase minoritaria (churn) tiene m√©tricas moderadas:
+ Precision ‚âà 0.57‚Äì0.59 ‚Üí el modelo acierta en ~57% de los casos que predice como churn.
+ Recall ‚âà 0.58‚Äì0.62 ‚Üí el modelo detecta ~60% de los churn reales.<br>

La matriz de confusi√≥n muestra que hay falsos negativos y falsos positivos, pero no extremos.


- Los modelos no est√°n sobreajustados: el rendimiento en test es coherente con la validaci√≥n cruzada.
- El rebalanceo con SMOTE ayud√≥ a mejorar la sensibilidad hacia la clase minoritaria.
- A√∫n hay espacio para mejorar recall sin sacrificar demasiado la precisi√≥n
- - -

### <font color='blue'>**5.- Interpretar los resultados, incluyendo la importancia de las variables**.

##### Importancia de variables en Logistic Regression

In [80]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

# Entrenar modelo
logreg_model_final = LogisticRegression(max_iter=1000, random_state=42)
logreg_model_final.fit(X_train_resampled, y_train_resampled)

# Extraer coeficientes
coefficients = logreg_model_final.coef_[0]
feature_names = X_train_resampled.columns

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Abs_Coefficient': np.abs(coefficients)
}).sort_values(by='Abs_Coefficient', ascending=False)

# Mostrar top 10
coef_df.head(10)

Feature  Coefficient  Abs_Coefficient
1                         customer.tenure    -1.581914         1.581914
3                   account.Charges.Total     0.865453         0.865453
11  internet.InternetService_Fibra √ìptica     0.807261         0.807261
26              account.Contract_Two year    -0.631468         0.631468
2                     Facturacion_Mensual    -0.594260         0.594260
4                         Cuentas_Diarias    -0.594260         0.594260
25              account.Contract_One year    -0.287498         0.287498
22               internet.StreamingTV_Yes     0.286415         0.286415
24           internet.StreamingMovies_Yes     0.256108         0.256108
10                phone.MultipleLines_Yes     0.233091         0.233091

##### Importancia de variables en Random Forest

In [81]:
from sklearn.ensemble import RandomForestClassifier

# Entrenar modelo
rf_model_final = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_final.fit(X_train_resampled, y_train_resampled)

# Extraer importancia
rf_importances = rf_model_final.feature_importances_
feature_names = X_train_resampled.columns

rf_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf_importances
}).sort_values(by='Importance', ascending=False)

# Mostrar top 10
rf_df.head(10)

Feature  Importance
1                          customer.tenure    0.152338
3                    account.Charges.Total    0.128153
4                          Cuentas_Diarias    0.096800
2                      Facturacion_Mensual    0.095489
26               account.Contract_Two year    0.062647
29  account.PaymentMethod_Electronic check    0.041713
11   internet.InternetService_Fibra √ìptica    0.040516
27            account.PaperlessBilling_Yes    0.034448
25               account.Contract_One year    0.032183
20                internet.TechSupport_Yes    0.028672

- Hay consistencia con Logistic Regression: tenure, Charges.Total, Contract_Two year, Facturacion_Mensual, InternetService_Fibra √ìptica aparecen en ambos modelos como relevantes.
- Random Forest no indica direcci√≥n (positivo/negativo), pero confirma la importancia estructural de estas variables.


##### Comparativa top 10: Logistic Regression vs Random Forest

| Variable                                 | LR: Coeficiente | RF: Importancia | Coinciden | Interpretaci√≥n clave                          |
|------------------------------------------|------------------|------------------|-----------|------------------------------------------------|
| customer.tenure                          | -1.58            | 0.152            | ‚úÖ        | M√°s antig√ºedad, menos churn                    |
| account.Charges.Total                    | +0.87            | 0.128            | ‚úÖ        | Cargos altos ‚Üí m√°s churn                       |
| internet.InternetService_Fibra √ìptica    | +0.81            | 0.041            | ‚úÖ        | Fibra √≥ptica ‚Üí m√°s churn                       |
| account.Contract_Two year                | -0.63            | 0.063            | ‚úÖ        | Contrato largo ‚Üí retenci√≥n                     |
| Facturacion_Mensual                      | -0.59            | 0.095            | ‚úÖ        | Menor facturaci√≥n ‚Üí menos churn                |
| Cuentas_Diarias                          | -0.59            | 0.097            | ‚úÖ        | M√°s actividad ‚Üí menos churn                    |
| account.Contract_One year                | -0.29            | 0.032            | ‚úÖ        | Contrato medio ‚Üí retenci√≥n                     |
| internet.StreamingTV_Yes                 | +0.29            | ‚Äî                | ‚ùå        | Streaming TV ‚Üí m√°s churn                       |
| internet.StreamingMovies_Yes             | +0.26            | ‚Äî                | ‚ùå        | Streaming Movies ‚Üí m√°s churn                   |
| phone.MultipleLines_Yes                  | +0.23            | ‚Äî                | ‚ùå        | M√∫ltiples l√≠neas ‚Üí m√°s churn                   |

Se evaluaron dos modelos predictivos de churn: Regresi√≥n Log√≠stica y Random Forest, con el objetivo de identificar las variables m√°s influyentes en la decisi√≥n de abandono del cliente.
Ambos modelos coincidieron en destacar:
- Antig√ºedad del cliente (tenure)
- Cargos acumulados (Charges.Total)
- Tipo de contrato (Contract_Two year, Contract_One year)
- Facturaci√≥n mensual
- Actividad diaria<br>

Estas variables deben considerarse prioritarias en estrategias de retenci√≥n.<br>

La regresi√≥n log√≠stica aporta direcci√≥n (efecto positivo o negativo), mientras que Random Forest confirma la relevancia estructural de las variables. La consistencia entre modelos refuerza la confiabilidad de los hallazgos.


### <font color='blue'>**6.- Crear una conclusi√≥n estrat√©gica se√±alando los principales factores que influyen en la cancelaci√≥n**.

##### **a) <u>Conclusi√≥n estrat√©gica sobre el churn**</u>

El an√°lisis de modelos predictivos revela un conjunto claro de factores que influyen significativamente en la cancelaci√≥n del servicio. Estos hallazgos permiten orientar acciones concretas para reducir el churn y mejorar la retenci√≥n.



##### **b) <u>Principales factores asociados al churn**</u>

- **Baja antig√ºedad del cliente (`tenure`)**  
  Clientes nuevos presentan mayor riesgo de cancelaci√≥n. Los primeros meses son cr√≠ticos para establecer valor y fidelidad.

- **Altos cargos acumulados (`Charges.Total`)**  
  La percepci√≥n de costos elevados est√° fuertemente asociada al abandono. Es clave revisar pol√≠ticas de facturaci√≥n y comunicar beneficios.

- **Tipo de contrato**  
  Contratos de mayor duraci√≥n (1 o 2 a√±os) se asocian con menor churn. Incentivar compromisos a largo plazo puede ser efectivo.

- **Facturaci√≥n mensual**  
  Menor facturaci√≥n mensual se vincula con menor churn, posiblemente por percepci√≥n de accesibilidad o valor.

- **Nivel de actividad (`Cuentas_Diarias`)**  
  Clientes m√°s activos tienden a permanecer. Fomentar el uso frecuente del servicio fortalece la relaci√≥n.

- **Tipo de servicio contratado**  
  Servicios como fibra √≥ptica y streaming se asocian con mayor churn. Es importante revisar la experiencia de usuario en estos segmentos.


##### **c) <u>Recomendaci√≥n estrat√©gica</u>**

Las acciones de retenci√≥n deben centrarse en:

- Fortalecer la experiencia de los clientes nuevos.
- Ofrecer incentivos por contratos largos.
- Revisar la estructura de precios y cargos acumulados.
- Promover el uso activo del servicio.
- Monitorear segmentos con servicios premium para detectar fricciones.